In [1]:
from nltk.tokenize import RegexpTokenizer
import os
import numpy as np
import logging
import operator
import copy
from collections import defaultdict,Counter
from random import shuffle
from pprint import pprint
import pickle
#from utils import get_files

global_subgraph_to_id_map = {}

logger = logging.getLogger()
logger.setLevel("INFO")

%run utils.ipynb

In [2]:
# # Set it manually for now
# corpus_dir = "/home/ipsita/BTP/graph2vec/data/kdd_datasets/ptc"
# output_dir = "/home/ipsita/BTP/graph2vec/embeddings"
# batch_size = 128
# epochs = 1000
# embedding_size = 1024
# num_negsample = 10
# learning_rate = 0.3
# wlk_h = 3
# label_filed_name = 'Label'
# class_labels_fname = '/home/ipsita/BTP/graph2vec/data/kdd_datasets/ptc.Labels'

# extn = 'g2v'+str(wlk_h)    # wlk_h is height to be considered

In [3]:
# #scan and load corpus
# graph_fname_list = get_files(corpus_dir, extn, 0)
# print('graph_fname_list', graph_fname_list[0])

# graph_name_to_id_map = {g: i for i, g in enumerate(graph_fname_list)}  # input layer of the skipgram network
# first2pairs = {k: graph_name_to_id_map[k] for k in list(graph_name_to_id_map)[:3]}
# print('**')
# print(first2pairs)

# id_to_graph_name_map = {i: g for g, i in graph_name_to_id_map.items()}


graph_fname_list /home/ipsita/BTP/graph2vec/data/kdd_datasets/ptc/0.gexf.g2v3
**
{'/home/ipsita/BTP/graph2vec/data/kdd_datasets/ptc/0.gexf.g2v3': 0, '/home/ipsita/BTP/graph2vec/data/kdd_datasets/ptc/1.gexf.g2v3': 1, '/home/ipsita/BTP/graph2vec/data/kdd_datasets/ptc/10.gexf.g2v3': 2}


In [4]:
# # create subgraph to id map with scan corpus
# # subgraph_to_id_map = self.scan_corpus()

# subgraphs = []
# for fname in graph_fname_list:
#     #print('fname=',fname)
#     subgraphs.extend([l.split()[0] for l in open(fname).readlines()])  # just take the first word of every sentence
#     #print(subgraphs)
#     #break
# subgraphs.append('UNK')

# ### subgraphs has repeating elements
# print(len(subgraphs))

# subgraph_to_freq_map = Counter(subgraphs)
# del subgraphs
# print(len(subgraph_to_freq_map))

34837
3804


In [5]:
# subgraph_to_id_map = {sg: i for i, sg in enumerate(subgraph_to_freq_map.keys())}  # output layer of the skipgram network
# print('subgraph_to_id_map', {k: subgraph_to_id_map[k] for k in list(subgraph_to_id_map)[:5]})

# # subgraph_to_freq_map  # to be used for negative sampling

# # self._subgraph_to_id_map = subgraph_to_id_map

# id_to_subgraph_map = {v:k for k,v in subgraph_to_id_map.items()}

# subgraphcount = sum(subgraph_to_freq_map.values()) #total num subgraphs in all graphs
# num_graphs    = len(graph_fname_list) #doc size
# num_subgraphs = len(subgraph_to_id_map) #vocab of word size

# subgraph_id_freq_map_as_list = [] #id of this list is the word id and value is the freq of word with corresponding word id
# for i in range(len(subgraph_to_freq_map)):
#     subgraph_id_freq_map_as_list.append(subgraph_to_freq_map[id_to_subgraph_map[i]])

# print(subgraph_id_freq_map_as_list[0:3])
# print("len=",len(subgraph_id_freq_map_as_list))

# logging.info('number of graphs: %d' % num_graphs)
# logging.info('subgraph vocabulary size: %d' % num_subgraphs)
# logging.info('total number of subgraphs to be trained: %d' % subgraphcount)

INFO:root:number of graphs: 344
INFO:root:subgraph vocabulary size: 3804
INFO:root:total number of subgraphs to be trained: 34837


subgraph_to_id_map {'0+3': 0, '1+3': 1, '2+27': 2, '3+53': 3, '0+1': 4}
[3851, 3428, 681]
len= 3804


In [6]:
# graph_ids_for_batch_traversal = list(range(num_graphs))  ###########################
# shuffle(graph_ids_for_batch_traversal)

In [14]:
# #generate batch here
# target_subgraph_ids = []
# context_subgraph_ids = []
# graph_index = 0
# subgraph_index = 2

# graph_name     = graph_fname_list[graph_ids_for_batch_traversal[graph_index]]
# graph_contents = open(graph_name).readlines()

# print(graph_name)
# print(len(graph_contents))
# print(graph_contents[10])

/home/ipsita/BTP/graph2vec/data/kdd_datasets/ptc/219.gexf.g2v3
112
2+14 2+35 1+8 3+1073



In [11]:
# while subgraph_index >= len(graph_contents)-2: #####
#     subgraph_index = 2
#     graph_index += 1
#     if graph_index == len(graph_fname_list):   #last graph so wrap around
#         graph_index = 0
#         np.random.shuffle(graph_ids_for_batch_traversal)
#         #epoch_flag = True

#     graph_name = graph_fname_list[graph_ids_for_batch_traversal[graph_index]]
#     graph_contents = open(graph_name).readlines()



In [15]:
# while len(context_subgraph_ids) < batch_size:

#     while subgraph_index >= len(graph_contents)-2: #########
#         subgraph_index = 2
#         graph_index += 1

#         if graph_index == len(graph_fname_list):
#             graph_index = 0
#             np.random.shuffle(graph_ids_for_batch_traversal)
#             #self.epoch_flag = True

#         graph_name = graph_fname_list[graph_ids_for_batch_traversal[graph_index]]
#         graph_contents = open(graph_name).readlines()

#     line_id = subgraph_index
#     context_subgraph_1 = graph_contents[line_id].split()[0]
#     context_subgraph_0 = graph_contents[line_id-1].split()[0]
#     context_subgraph_m1 = graph_contents[line_id-2].split()[0]
#     context_subgraph_2 = graph_contents[line_id+1].split()[0]
#     context_subgraph_3 = graph_contents[line_id+2].split()[0]
#     target_graph = graph_name

#     print('target =', context_subgraph_1, 'graph=',  target_graph)
#     print('context =', context_subgraph_m1, context_subgraph_0, context_subgraph_2, context_subgraph_3)
#     context_subgraph_ids.append([graph_name_to_id_map[target_graph],
#                                  subgraph_to_id_map[context_subgraph_m1],
#                                  subgraph_to_id_map[context_subgraph_0],
#                                  subgraph_to_id_map[context_subgraph_2],
#                                  subgraph_to_id_map[context_subgraph_3]
#                                 ])
#     target_subgraph_ids.append(subgraph_to_id_map[context_subgraph_1])

#     subgraph_index += 1

target = 2+14 graph= /home/ipsita/BTP/graph2vec/data/kdd_datasets/ptc/219.gexf.g2v3
context = 0+3 1+3 3+43 0+3
target = 3+43 graph= /home/ipsita/BTP/graph2vec/data/kdd_datasets/ptc/219.gexf.g2v3
context = 1+3 2+14 0+3 1+3
target = 0+3 graph= /home/ipsita/BTP/graph2vec/data/kdd_datasets/ptc/219.gexf.g2v3
context = 2+14 3+43 1+3 2+14
target = 1+3 graph= /home/ipsita/BTP/graph2vec/data/kdd_datasets/ptc/219.gexf.g2v3
context = 3+43 0+3 2+14 3+45
target = 2+14 graph= /home/ipsita/BTP/graph2vec/data/kdd_datasets/ptc/219.gexf.g2v3
context = 0+3 1+3 3+45 0+3
target = 3+45 graph= /home/ipsita/BTP/graph2vec/data/kdd_datasets/ptc/219.gexf.g2v3
context = 1+3 2+14 0+3 1+3
target = 0+3 graph= /home/ipsita/BTP/graph2vec/data/kdd_datasets/ptc/219.gexf.g2v3
context = 2+14 3+45 1+3 2+14
target = 1+3 graph= /home/ipsita/BTP/graph2vec/data/kdd_datasets/ptc/219.gexf.g2v3
context = 3+45 0+3 2+14 3+45
target = 2+14 graph= /home/ipsita/BTP/graph2vec/data/kdd_datasets/ptc/219.gexf.g2v3
context = 0+3 1+3 3+45 0

In [19]:
# print(target_subgraph_ids,"len=",len(target_subgraph_ids))
# #print(context_subgraph_ids,"len=",len(context_subgraph_ids))
# print(graph_index)
# print(subgraph_index)  # 92 + 36 = 128

[31, 89, 0, 1, 31, 867, 0, 1, 31, 867, 0, 1, 31, 89, 0, 1, 31, 89, 0, 1, 31, 867, 0, 1, 31, 867, 0, 1, 31, 89, 4, 1576, 1966, 1967, 41, 42, 1579, 1968, 4, 157, 1969, 1970, 41, 42, 1579, 1968, 4, 47, 72, 1965, 4, 14, 1361, 1971, 4, 38, 871, 1396, 4, 47, 875, 1636, 4, 47, 875, 1636, 41, 42, 43, 877, 4, 14, 1361, 1971, 4, 47, 72, 1965, 4, 47, 875, 1636, 4, 47, 72, 1965, 41, 42, 43, 877, 4, 38, 871, 1396, 4, 47, 72, 1965, 4, 47, 875, 1636, 0, 1, 175, 1972, 0, 1, 31, 128, 0, 33, 246, 272, 0, 33, 246, 272, 0, 1, 31, 128, 0, 33, 246, 272, 0, 1] len= 128
1
22


In [24]:
# target_subgraph_ids2 = []
# context_subgraph_ids2 = []
# index_shuf = list(range(len(target_subgraph_ids)))

# shuffle(index_shuf)
# for i in index_shuf:
#     target_subgraph_ids2.append(target_subgraph_ids[i])
#     context_subgraph_ids2.append(context_subgraph_ids[i])

# #target_context_pairs = zip(target_graph_ids, context_subgraph_ids)
# #shuffle(target_context_pairs)
# #target_graph_ids, context_subgraph_ids = zip(*target_context_pairs)

# target_subgraph_ids = np.array(target_subgraph_ids2, dtype=np.int32)
# context_subgraph_ids = np.array(context_subgraph_ids2, dtype=np.int32)

# targetword_outputs = np.reshape(target_subgraph_ids, [len(target_subgraph_ids), 1])

# print(targetword_outputs[0])
# print(context_subgraph_ids[0])

[1636]
[134  47 875  41  42]


# PV-DM Model

In [18]:
# target_subgraph_ids = []
# context_subgraph_ids = []
# graph_index = 0
# subgraph_index = 1

# graph_name     = graph_fname_list[graph_ids_for_batch_traversal[graph_index]]
# graph_contents = open(graph_name).readlines()

# print(graph_name)
# print(len(graph_contents))

/home/ipsita/BTP/graph2vec/data/kdd_datasets/ptc/40.gexf.g2v3
80


In [19]:
# while subgraph_index >= len(graph_contents)-1:
#     subgraph_index = 1
#     graph_index += 1
    
#     if graph_index == len(graph_fname_list):   #last graph so wrap around
#         graph_index = 0
#         np.random.shuffle(graph_ids_for_batch_traversal)
#         #epoch_flag = True
        
#     graph_name = graph_fname_list[graph_ids_for_batch_traversal[graph_index]]
#     graph_contents = open(graph_name).readlines()

In [20]:
# while len(context_subgraph_ids) < batch_size:  #########
    
#         #if(line_id == 0):
#         #    line_id = 1
#         line_id = subgraph_index
#         context_subgraph_1 = graph_contents[line_id].split()[0]
#         context_subgraph_0 = graph_contents[line_id-1].split()[0]
#         context_subgraph_2 = graph_contents[line_id+1].split()[0]
        
#         target_graph = graph_name
#         print('context:', context_subgraph_0,context_subgraph_1,context_subgraph_2)
#         print('target_graph=',target_graph)

#         #context_subgraph_ids.append(subgraph_to_id_map[context_subgraph])
#         #target_graph_ids.append(graph_name_to_id_map[target_graph])
        
#         context_subgraph_ids.append([graph_name_to_id_map[target_graph],
#                                      subgraph_to_id_map[context_subgraph_0],
#                                      subgraph_to_id_map[context_subgraph_2]
#                                     ])
        
#         target_subgraph_ids.append(subgraph_to_id_map[context_subgraph_1])

#         subgraph_index += 1
#         while subgraph_index == len(graph_contents)-1:
#             subgraph_index = 1
#             graph_index += 1

#             if graph_index == len(graph_fname_list):
#                 graph_index = 0
#                 np.random.shuffle(graph_ids_for_batch_traversal)
#                 #self.epoch_flag = True

#             graph_name = graph_fname_list[graph_ids_for_batch_traversal[graph_index]]
#             graph_contents = open(graph_name).readlines()

# print(len(context_subgraph_ids))

context: 0+3 1+3 2+27
target_graph= /home/ipsita/BTP/graph2vec/data/kdd_datasets/ptc/40.gexf.g2v3
context: 1+3 2+27 3+139
target_graph= /home/ipsita/BTP/graph2vec/data/kdd_datasets/ptc/40.gexf.g2v3
context: 2+27 3+139 0+3
target_graph= /home/ipsita/BTP/graph2vec/data/kdd_datasets/ptc/40.gexf.g2v3
context: 3+139 0+3 1+3
target_graph= /home/ipsita/BTP/graph2vec/data/kdd_datasets/ptc/40.gexf.g2v3
context: 0+3 1+3 2+109
target_graph= /home/ipsita/BTP/graph2vec/data/kdd_datasets/ptc/40.gexf.g2v3
context: 1+3 2+109 3+1409
target_graph= /home/ipsita/BTP/graph2vec/data/kdd_datasets/ptc/40.gexf.g2v3
context: 2+109 3+1409 0+3
target_graph= /home/ipsita/BTP/graph2vec/data/kdd_datasets/ptc/40.gexf.g2v3
context: 3+1409 0+3 1+3
target_graph= /home/ipsita/BTP/graph2vec/data/kdd_datasets/ptc/40.gexf.g2v3
context: 0+3 1+3 2+27
target_graph= /home/ipsita/BTP/graph2vec/data/kdd_datasets/ptc/40.gexf.g2v3
context: 1+3 2+27 3+139
target_graph= /home/ipsita/BTP/graph2vec/data/kdd_datasets/ptc/40.gexf.g2v3
co

In [27]:
#context_subgraph_ids

In [ ]:
#target_subgraph_ids

In [31]:
# target_subgraph_ids2 = []
# context_subgraph_ids2 = []
# index_shuf = list(range(len(target_subgraph_ids)))

# shuffle(index_shuf)
# for i in index_shuf:
#     target_subgraph_ids2.append(target_subgraph_ids[i])
#     context_subgraph_ids2.append(context_subgraph_ids[i])
    
# target_subgraph_ids = np.array(target_subgraph_ids2, dtype=np.int32)
# context_subgraph_ids = np.array(context_subgraph_ids2, dtype=np.int32)

# targetword_outputs = np.reshape(target_subgraph_ids, [len(target_subgraph_ids), 1])

In [34]:
# print(np.shape(target_subgraph_ids))
# print(np.shape(context_subgraph_ids))
# print(np.shape(targetword_outputs))

(128,)
(128, 3)
(128, 1)


In [2]:
class Corpus(object):
    
    def __init__(self, win_size, corpus_folder=None, extn='WL2', max_files=0):
        
        assert corpus_folder != None, "please specify the corpus folder"
        self.corpus_folder = corpus_folder
        self.win_size = win_size
        self.subgraph_index = 1  #########
        self.graph_index = 0
        self.epoch_flag = 0
        self.max_files = max_files
        self.graph_ids_for_batch_traversal = []
        self.extn = extn


    def scan_corpus(self):

        subgraphs = []
        for fname in self.graph_fname_list:
            subgraphs.extend(
                [l.split()[0] for l in open(fname).readlines()])  # just take the first word of every sentence
        subgraphs.append('UNK')

        subgraph_to_freq_map = Counter(subgraphs)
        del subgraphs

        subgraph_to_id_map = {sg: i for i, sg in
                              enumerate(subgraph_to_freq_map.keys())}  # output layer of the skipgram network

        self._subgraph_to_freq_map = subgraph_to_freq_map  # to be used for negative sampling
        self._subgraph_to_id_map = subgraph_to_id_map
        self._id_to_subgraph_map = {v:k for k,v in subgraph_to_id_map.items()}
        self._subgraphcount = sum(subgraph_to_freq_map.values()) #total num subgraphs in all graphs

        self.num_graphs = len(self.graph_fname_list) #doc size
        self.num_subgraphs = len(subgraph_to_id_map) #vocab of word size

        self.subgraph_id_freq_map_as_list = [] #id of this list is the word id and value is the freq of word with corresponding word id
        for i in range(len(self._subgraph_to_freq_map)):
            self.subgraph_id_freq_map_as_list.append(self._subgraph_to_freq_map[self._id_to_subgraph_map[i]])

        return self._subgraph_to_id_map


    def scan_and_load_corpus(self, graph_list):
        
        #self.graph_fname_list = get_files(self.corpus_folder, extn=self.extn, max_files=self.max_files)
        self.graph_fname_list = graph_list
        self._graph_name_to_id_map = {g: i for i, g in
                                      enumerate(self.graph_fname_list)}  # input layer of the skipgram network
        self._id_to_graph_name_map = {i: g for g, i in self._graph_name_to_id_map.items()}
        subgraph_to_id_map = self.scan_corpus()
        
        ### Save subgraph2id map in file
        with open('/home/ipsita/BTP/graph2vec/tmp/for_gui/subgraph2id.p', 'wb') as fp:
            pickle.dump(subgraph_to_id_map, fp, protocol=pickle.HIGHEST_PROTOCOL)
        
        logging.info('number of graphs: %d' % self.num_graphs)
        logging.info('subgraph vocabulary size: %d' % self.num_subgraphs)
        logging.info('total number of subgraphs to be trained: %d' % self._subgraphcount)
        
        print('number of graphs: %d' % self.num_graphs)
        print('subgraph vocabulary size: %d' % self.num_subgraphs)
        print('total number of subgraphs to be trained: %d' % self._subgraphcount)

        self.graph_ids_for_batch_traversal = list(range(self.num_graphs))  ###########################
        shuffle(self.graph_ids_for_batch_traversal)
        
        global global_subgraph_to_id_map
        global_subgraph_to_id_map = {}
        global_subgraph_to_id_map = copy.deepcopy(subgraph_to_id_map)
        
        return self.subgraph_id_freq_map_as_list
        
    def scan_and_load_corpus_for_test(self, graph_list):
        self.graph_fname_list = graph_list
        self._graph_name_to_id_map = {g: i for i, g in enumerate(self.graph_fname_list)}
        self._id_to_graph_name_map = {i: g for g, i in self._graph_name_to_id_map.items()}
        
        self.num_graphs = len(self.graph_fname_list)
        self.graph_ids_for_batch_traversal = list(range(self.num_graphs))  ###########################
        shuffle(self.graph_ids_for_batch_traversal)
        
    def generate_batch_from_file_for_test(self, batch_size):
        global global_subgraph_to_id_map
        
        with open('/home/ipsita/BTP/graph2vec/tmp/for_gui/subgraph2id.p', 'rb') as fp:
            global_subgraph_to_id_map = pickle.load(fp)
            
        target_subgraph_ids = []
        context_subgraph_ids = []
        
        graph_name     = self.graph_fname_list[self.graph_ids_for_batch_traversal[self.graph_index]]
        graph_contents = open(graph_name).readlines()
        while self.subgraph_index >= len(graph_contents)-self.win_size:
            self.subgraph_index = self.win_size
            self.graph_index += 1

            if self.graph_index == len(self.graph_fname_list):   #last graph so wrap around
                self.graph_index = 0
                np.random.shuffle(self.graph_ids_for_batch_traversal)
                self.epoch_flag = True

            graph_name = self.graph_fname_list[self.graph_ids_for_batch_traversal[self.graph_index]]
            graph_contents = open(graph_name).readlines()
            
        while len(context_subgraph_ids) < batch_size:

            while self.subgraph_index >= len(graph_contents)-self.win_size:
                self.subgraph_index = self.win_size
                self.graph_index += 1

                if self.graph_index == len(self.graph_fname_list):
                    self.graph_index = 0
                    np.random.shuffle(self.graph_ids_for_batch_traversal)
                    self.epoch_flag = True

                graph_name = self.graph_fname_list[self.graph_ids_for_batch_traversal[self.graph_index]]
                graph_contents = open(graph_name).readlines()
            
            line_id = self.subgraph_index
            #context_subgraph_1 = graph_contents[line_id].split()[0]
            #context_subgraph_0 = graph_contents[line_id-1].split()[0]
            #context_subgraph_2 = graph_contents[line_id+1].split()[0]
            
            target_graph = graph_name
            context_subgraph_list = []
            context_subgraph_list.append(self._graph_name_to_id_map[target_graph])
            for i in range(-self.win_size, self.win_size + 1):
                if i != 0:
                    #print("win size = ", self.win_size, "i=", i)
                    k = graph_contents[line_id + i].split()[0]
                    if k in global_subgraph_to_id_map:
                        context_subgraph_list.append(global_subgraph_to_id_map[graph_contents[line_id + i].split()[0]] )
                    else:
                        context_subgraph_list.append(global_subgraph_to_id_map['UNK'])
             
            #print("mini list", context_subgraph_list)
            context_subgraph_ids.append(context_subgraph_list)
            
            #context_subgraph_ids.append([self._graph_name_to_id_map[target_graph],
            #                             self._subgraph_to_id_map[context_subgraph_0],
            #                             self._subgraph_to_id_map[context_subgraph_2]
            #                            ])
            #target_subgraph_ids.append(self._subgraph_to_id_map[context_subgraph_1])
            if graph_contents[line_id].split()[0] in global_subgraph_to_id_map:
                target_subgraph_ids.append(global_subgraph_to_id_map[graph_contents[line_id].split()[0]])
            else:
                target_subgraph_ids.append(global_subgraph_to_id_map['UNK'])
            
            self.subgraph_index += 1
        
        #print("context_subgraph_ids shape =",np.shape(context_subgraph_ids) ,"\n", context_subgraph_ids)
        target_subgraph_ids2 = []
        context_subgraph_ids2 = []
        index_shuf = list(range(len(target_subgraph_ids)))

        shuffle(index_shuf)
        for i in index_shuf:
            target_subgraph_ids2.append(target_subgraph_ids[i])
            context_subgraph_ids2.append(context_subgraph_ids[i])

        #target_context_pairs = zip(target_graph_ids, context_subgraph_ids)
        #shuffle(target_context_pairs)
        #target_graph_ids, context_subgraph_ids = zip(*target_context_pairs)
        
        #context_subgraph_ids = []
        target_subgraph_ids = np.array(target_subgraph_ids2, dtype=np.int32)
        context_subgraph_ids = np.array(context_subgraph_ids2, dtype=np.int32)

        targetword_outputs = np.reshape(target_subgraph_ids, [len(target_subgraph_ids), 1])
        #contextword_outputs = np.reshape(context_subgraph_ids, [len(context_subgraph_ids), 1])

        #np.savetxt("batch.txt", (context_subgraph_ids, targetword_outputs), fmt="%d")
        #print("shape of batch context:", np.shape(context_subgraph_ids), "target", np.shape(targetword_outputs))
        return context_subgraph_ids, targetword_outputs
    
    def generate_batch_from_file(self, batch_size):
        target_subgraph_ids = []
        context_subgraph_ids = []
        
        graph_name     = self.graph_fname_list[self.graph_ids_for_batch_traversal[self.graph_index]]
        graph_contents = open(graph_name).readlines()
        while self.subgraph_index >= len(graph_contents)-self.win_size:
            self.subgraph_index = self.win_size
            self.graph_index += 1

            if self.graph_index == len(self.graph_fname_list):   #last graph so wrap around
                self.graph_index = 0
                np.random.shuffle(self.graph_ids_for_batch_traversal)
                self.epoch_flag = True

            graph_name = self.graph_fname_list[self.graph_ids_for_batch_traversal[self.graph_index]]
            graph_contents = open(graph_name).readlines()
            
        while len(context_subgraph_ids) < batch_size:
            

            while self.subgraph_index >= len(graph_contents)-self.win_size:
                self.subgraph_index = self.win_size
                self.graph_index += 1

                if self.graph_index == len(self.graph_fname_list):
                    self.graph_index = 0
                    np.random.shuffle(self.graph_ids_for_batch_traversal)
                    self.epoch_flag = True

                graph_name = self.graph_fname_list[self.graph_ids_for_batch_traversal[self.graph_index]]
                graph_contents = open(graph_name).readlines()
            
            line_id = self.subgraph_index
            #context_subgraph_1 = graph_contents[line_id].split()[0]
            #context_subgraph_0 = graph_contents[line_id-1].split()[0]
            #context_subgraph_2 = graph_contents[line_id+1].split()[0]
            
            target_graph = graph_name
            context_subgraph_list = []
            context_subgraph_list.append(self._graph_name_to_id_map[target_graph])
            for i in range(-self.win_size, self.win_size + 1):
                if i != 0:
                    #print("win size = ", self.win_size, "i=", i)
                    context_subgraph_list.append( self._subgraph_to_id_map[graph_contents[line_id + i].split()[0]] )
             
            #print("mini list", context_subgraph_list)
            context_subgraph_ids.append(context_subgraph_list)
            
            #context_subgraph_ids.append([self._graph_name_to_id_map[target_graph],
            #                             self._subgraph_to_id_map[context_subgraph_0],
            #                             self._subgraph_to_id_map[context_subgraph_2]
            #                            ])
            #target_subgraph_ids.append(self._subgraph_to_id_map[context_subgraph_1])
            target_subgraph_ids.append(self._subgraph_to_id_map[graph_contents[line_id].split()[0]])
            
            self.subgraph_index += 1
        
        #print("context_subgraph_ids shape =",np.shape(context_subgraph_ids) ,"\n", context_subgraph_ids)
        target_subgraph_ids2 = []
        context_subgraph_ids2 = []
        index_shuf = list(range(len(target_subgraph_ids)))

        shuffle(index_shuf)
        for i in index_shuf:
            target_subgraph_ids2.append(target_subgraph_ids[i])
            context_subgraph_ids2.append(context_subgraph_ids[i])

        #target_context_pairs = zip(target_graph_ids, context_subgraph_ids)
        #shuffle(target_context_pairs)
        #target_graph_ids, context_subgraph_ids = zip(*target_context_pairs)
        
        #context_subgraph_ids = []
        target_subgraph_ids = np.array(target_subgraph_ids2, dtype=np.int32)
        context_subgraph_ids = np.array(context_subgraph_ids2, dtype=np.int32)

        targetword_outputs = np.reshape(target_subgraph_ids, [len(target_subgraph_ids), 1])
        #contextword_outputs = np.reshape(context_subgraph_ids, [len(context_subgraph_ids), 1])

        #np.savetxt("batch.txt", (context_subgraph_ids, targetword_outputs), fmt="%d")
        #print("shape of batch context:", np.shape(context_subgraph_ids), "target", np.shape(targetword_outputs))
        return context_subgraph_ids, targetword_outputs

In [22]:
# OLDER SKIPGRAM MODEL CODE
#     def generate_batch_from_file(self, batch_size):
#         target_graph_ids = []
#         context_subgraph_ids = []

#         graph_name = self.graph_fname_list[self.graph_ids_for_batch_traversal[self.graph_index]]
#         graph_contents = open(graph_name).readlines()
#         while self.subgraph_index >= len(graph_contents):
#             self.subgraph_index = 0
#             self.graph_index += 1
#             if self.graph_index == len(self.graph_fname_list):
#                 self.graph_index = 0
#                 np.random.shuffle(self.graph_ids_for_batch_traversal)
#                 self.epoch_flag = True
#             graph_name = self.graph_fname_list[self.graph_ids_for_batch_traversal[self.graph_index]]
#             graph_contents = open(graph_name).readlines()

#         while len(context_subgraph_ids) < batch_size:
#             line_id = self.subgraph_index
#             context_subgraph = graph_contents[line_id].split()[0]
#             target_graph = graph_name

#             context_subgraph_ids.append(self._subgraph_to_id_map[context_subgraph])
#             target_graph_ids.append(self._graph_name_to_id_map[target_graph])

#             self.subgraph_index+=1
#             while self.subgraph_index == len(graph_contents):
#                 self.subgraph_index = 0
#                 self.graph_index += 1
#                 if self.graph_index == len(self.graph_fname_list):
#                     self.graph_index = 0
#                     np.random.shuffle(self.graph_ids_for_batch_traversal)
#                     self.epoch_flag = True

#                 graph_name = self.graph_fname_list[self.graph_ids_for_batch_traversal[self.graph_index]]
#                 graph_contents = open(graph_name).readlines()

        
#         target_graph_ids2 = []
#         context_subgraph_ids2 = []
#         index_shuf = list(range(len(target_graph_ids)))
        
#         shuffle(index_shuf)
#         for i in index_shuf:
#             target_graph_ids2.append(target_graph_ids[i])
#             context_subgraph_ids2.append(context_subgraph_ids[i])
        
#         #target_context_pairs = zip(target_graph_ids, context_subgraph_ids)
#         #shuffle(target_context_pairs)
#         #target_graph_ids, context_subgraph_ids = zip(*target_context_pairs)

#         target_graph_ids = np.array(target_graph_ids2, dtype=np.int32)
#         context_subgraph_ids = np.array(context_subgraph_ids2, dtype=np.int32)

#         contextword_outputs = np.reshape(context_subgraph_ids, [len(context_subgraph_ids), 1])
        
#         np.savetxt("batch.txt", (target_graph_ids,context_subgraph_ids), fmt="%d")
      

#         return target_graph_ids,contextword_outputs